# 1. 웹크롤링

In [1]:
import sys # 시스템
import os  # 시스템

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import subprocess
import shutil

try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제
except FileNotFoundError:
    pass

subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(10)

## 1.1 아이템 크롤링

In [24]:
%%time


# 제품 url들 수집

name_list = []
num_list =[]
rating_list=[]
rev_url_list=[]

try:
    for j in range(0,3):
        driver.get('https://kr.iherb.com/c/mct-oil?p={}'.format(j+1))
        for i in range(0,24):

            item = "div.product-inner.product-inner-wide > div.absolute-link-wrapper > a"
            item_raw = driver.find_elements_by_css_selector(item)

            rate=".rating-count"
            rating_raw =driver.find_elements_by_css_selector(rate)

            # 상품번호 크롤링
            num = item_raw[i].get_attribute('data-product-id')
            num_list.append(num)
            # 제목 크롤링 
            name = item_raw[i].get_attribute('title')
            name_list.append(name)

            #평점 크롤링
            rating=rating_raw[i].get_attribute('title')
            rating_list.append(rating)
            #리뷰창 크롤링
            review=rating_raw[i].get_attribute('href')
            rev_url_list.append(review)

            print(i,name,rating)
            time.sleep(2)
        
except:
    driver.close()

0 California Gold Nutrition, 유기농 MCT 오일, 946ml(32fl oz) 4.8/5 - 5969 평가
1 California Gold Nutrition, 유기농 MCT 오일, 355ml(12fl oz) 4.8/5 - 5969 평가
2 Nature's Way, 유기농 MCT 오일, 480ml(16fl oz) 4.7/5 - 6447 평가
3 Nature's Way, 유기농 MCT 오일, 887ml(30fl oz) 4.7/5 - 6447 평가
4 Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl oz) 4.7/5 - 719 평가
5 Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml) 4.7/5 - 2215 평가
6 NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz) 4.6/5 - 1921 평가
7 NOW Foods, 카프릴산, 600mg, 소프트젤 100정 4.8/5 - 3600 평가
8 Sports Research, MCT 오일, 1,000mg, 소프트젤 120정 4.7/5 - 391 평가
9 Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10.6oz) 4.7/5 - 273 평가
10 NOW Foods, Sports, 중쇄중성지방 오일, 946ml(32fl oz) 4.7/5 - 710 평가
11 Sports Research, MCT 오일, 무 맛, 16 oz (473 ml) 4.7/5 - 2215 평가
12 Jarrow Formulas, MCT 오일, 무맛, 591ml(20fl oz) 4.6/5 - 1563 평가
13 Nutiva, 100% 유기농 코코넛 MCT 오일, 무맛, 473ml(16fl oz) 4.7/5 - 489 평가
14 Garden of Life, 박사가 만든 케토, 유기농 MCT 분말, 300g(10.58oz) 4.7/5 - 477 평가
15 Garden of Life, 닥터 포뮬레이티드 두뇌 

In [25]:
df1 = pd.DataFrame({'name':name_list, 'item_id':num_list})
df1

,name,item_id
0,"California Gold Nutrition, 유기농 MCT 오일, 946ml(3...",99739
1,"California Gold Nutrition, 유기농 MCT 오일, 355ml(1...",99738
2,"Nature's Way, 유기농 MCT 오일, 480ml(16fl oz)",66202
3,"Nature's Way, 유기농 MCT 오일, 887ml(30fl oz)",77645
4,"Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl...",84966
5,"Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml)",71107
6,"NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz)",72853
7,"NOW Foods, 카프릴산, 600mg, 소프트젤 100정",13376
8,"Sports Research, MCT 오일, 1,000mg, 소프트젤 120정",75244
9,"Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10...",84970


In [26]:
df2 = pd.DataFrame({"평점":rating_list,"리뷰url":rev_url_list})
df2

,평점,리뷰url
0,4.8/5 - 5969 평가,https://kr.iherb.com/r/california-gold-nutriti...
1,4.8/5 - 5969 평가,https://kr.iherb.com/r/california-gold-nutriti...
2,4.7/5 - 6447 평가,https://kr.iherb.com/r/nature-s-way-organic-mc...
3,4.7/5 - 6447 평가,https://kr.iherb.com/r/nature-s-way-organic-mc...
4,4.7/5 - 719 평가,https://kr.iherb.com/r/sports-research-organic...
5,4.7/5 - 2215 평가,https://kr.iherb.com/r/sports-research-mct-oil...
6,4.6/5 - 1921 평가,https://kr.iherb.com/r/now-foods-sports-mct-oi...
7,4.8/5 - 3600 평가,https://kr.iherb.com/r/now-foods-caprylic-acid...
8,4.7/5 - 391 평가,https://kr.iherb.com/r/sports-research-mct-oil...
9,4.7/5 - 273 평가,https://kr.iherb.com/r/sports-research-organic...


In [27]:
result = pd.concat([df1,df2],axis=1)
result.head(10)

,name,item_id,평점,리뷰url
0,"California Gold Nutrition, 유기농 MCT 오일, 946ml(3...",99739,4.8/5 - 5969 평가,https://kr.iherb.com/r/california-gold-nutriti...
1,"California Gold Nutrition, 유기농 MCT 오일, 355ml(1...",99738,4.8/5 - 5969 평가,https://kr.iherb.com/r/california-gold-nutriti...
2,"Nature's Way, 유기농 MCT 오일, 480ml(16fl oz)",66202,4.7/5 - 6447 평가,https://kr.iherb.com/r/nature-s-way-organic-mc...
3,"Nature's Way, 유기농 MCT 오일, 887ml(30fl oz)",77645,4.7/5 - 6447 평가,https://kr.iherb.com/r/nature-s-way-organic-mc...
4,"Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl...",84966,4.7/5 - 719 평가,https://kr.iherb.com/r/sports-research-organic...
5,"Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml)",71107,4.7/5 - 2215 평가,https://kr.iherb.com/r/sports-research-mct-oil...
6,"NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz)",72853,4.6/5 - 1921 평가,https://kr.iherb.com/r/now-foods-sports-mct-oi...
7,"NOW Foods, 카프릴산, 600mg, 소프트젤 100정",13376,4.8/5 - 3600 평가,https://kr.iherb.com/r/now-foods-caprylic-acid...
8,"Sports Research, MCT 오일, 1,000mg, 소프트젤 120정",75244,4.7/5 - 391 평가,https://kr.iherb.com/r/sports-research-mct-oil...
9,"Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10...",84970,4.7/5 - 273 평가,https://kr.iherb.com/r/sports-research-organic...


### 1.1.1 데이터전처리

In [29]:
평점 = result.평점.str.split(' -').str[0].tolist()
평점수 = result.평점.str.split(' -').str[1].tolist()
result["평점"]=평점
result["평점수"]=평점수
result

,name,item_id,평점,리뷰url,평점수
0,"California Gold Nutrition, 유기농 MCT 오일, 946ml(3...",99739,4.8/5,https://kr.iherb.com/r/california-gold-nutriti...,5969 평가
1,"California Gold Nutrition, 유기농 MCT 오일, 355ml(1...",99738,4.8/5,https://kr.iherb.com/r/california-gold-nutriti...,5969 평가
2,"Nature's Way, 유기농 MCT 오일, 480ml(16fl oz)",66202,4.7/5,https://kr.iherb.com/r/nature-s-way-organic-mc...,6447 평가
3,"Nature's Way, 유기농 MCT 오일, 887ml(30fl oz)",77645,4.7/5,https://kr.iherb.com/r/nature-s-way-organic-mc...,6447 평가
4,"Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl...",84966,4.7/5,https://kr.iherb.com/r/sports-research-organic...,719 평가
5,"Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml)",71107,4.7/5,https://kr.iherb.com/r/sports-research-mct-oil...,2215 평가
6,"NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz)",72853,4.6/5,https://kr.iherb.com/r/now-foods-sports-mct-oi...,1921 평가
7,"NOW Foods, 카프릴산, 600mg, 소프트젤 100정",13376,4.8/5,https://kr.iherb.com/r/now-foods-caprylic-acid...,3600 평가
8,"Sports Research, MCT 오일, 1,000mg, 소프트젤 120정",75244,4.7/5,https://kr.iherb.com/r/sports-research-mct-oil...,391 평가
9,"Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10...",84970,4.7/5,https://kr.iherb.com/r/sports-research-organic...,273 평가


In [30]:
평점= result.평점.str.split('/').str[0].tolist()
평점수=result.평점수.str.split(" ").str[1].tolist()

In [31]:
result["평점"]=평점
result["평점수"]=평점수
result

,name,item_id,평점,리뷰url,평점수
0,"California Gold Nutrition, 유기농 MCT 오일, 946ml(3...",99739,4.8,https://kr.iherb.com/r/california-gold-nutriti...,5969
1,"California Gold Nutrition, 유기농 MCT 오일, 355ml(1...",99738,4.8,https://kr.iherb.com/r/california-gold-nutriti...,5969
2,"Nature's Way, 유기농 MCT 오일, 480ml(16fl oz)",66202,4.7,https://kr.iherb.com/r/nature-s-way-organic-mc...,6447
3,"Nature's Way, 유기농 MCT 오일, 887ml(30fl oz)",77645,4.7,https://kr.iherb.com/r/nature-s-way-organic-mc...,6447
4,"Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl...",84966,4.7,https://kr.iherb.com/r/sports-research-organic...,719
5,"Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml)",71107,4.7,https://kr.iherb.com/r/sports-research-mct-oil...,2215
6,"NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz)",72853,4.6,https://kr.iherb.com/r/now-foods-sports-mct-oi...,1921
7,"NOW Foods, 카프릴산, 600mg, 소프트젤 100정",13376,4.8,https://kr.iherb.com/r/now-foods-caprylic-acid...,3600
8,"Sports Research, MCT 오일, 1,000mg, 소프트젤 120정",75244,4.7,https://kr.iherb.com/r/sports-research-mct-oil...,391
9,"Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10...",84970,4.7,https://kr.iherb.com/r/sports-research-organic...,273


In [32]:
result["평점"][47]="4.7"
result["평점수"][47]="584"
result.isnull().sum()

name       0
item_id    0
평점         0
리뷰url      0
평점수        0
dtype: int64

In [33]:
result.dtypes

name       object
item_id    object
평점         object
리뷰url      object
평점수        object
dtype: object

In [34]:
result["평점"]=pd.to_numeric(result["평점"]) 
result["평점수"]=pd.to_numeric(result["평점수"]) 

result.dtypes

name        object
item_id     object
평점         float64
리뷰url       object
평점수          int64
dtype: object

In [35]:
#중복제거
result=result.drop_duplicates(['평점수'],keep = 'first')
result

,name,item_id,평점,리뷰url,평점수
0,"California Gold Nutrition, 유기농 MCT 오일, 946ml(3...",99739,4.8,https://kr.iherb.com/r/california-gold-nutriti...,5969
2,"Nature's Way, 유기농 MCT 오일, 480ml(16fl oz)",66202,4.7,https://kr.iherb.com/r/nature-s-way-organic-mc...,6447
4,"Sports Research, 유기농 MCT C8 오일, 무맛, 473ml(16fl...",84966,4.7,https://kr.iherb.com/r/sports-research-organic...,719
5,"Sports Research, MCT 오일, 무맛, 32 액량 온스 (946 ml)",71107,4.7,https://kr.iherb.com/r/sports-research-mct-oil...,2215
6,"NOW Foods, Sports, MCT 오일, 무맛, 473ml(16fl oz)",72853,4.6,https://kr.iherb.com/r/now-foods-sports-mct-oi...,1921
7,"NOW Foods, 카프릴산, 600mg, 소프트젤 100정",13376,4.8,https://kr.iherb.com/r/now-foods-caprylic-acid...,3600
8,"Sports Research, MCT 오일, 1,000mg, 소프트젤 120정",75244,4.7,https://kr.iherb.com/r/sports-research-mct-oil...,391
9,"Sports Research, 유기농 중쇄중성지방 오일 가루, 무맛, 300g(10...",84970,4.7,https://kr.iherb.com/r/sports-research-organic...,273
10,"NOW Foods, Sports, 중쇄중성지방 오일, 946ml(32fl oz)",13585,4.7,https://kr.iherb.com/r/now-foods-sports-mct-oi...,710
12,"Jarrow Formulas, MCT 오일, 무맛, 591ml(20fl oz)",68705,4.6,https://kr.iherb.com/r/jarrow-formulas-mct-oil...,1563


In [36]:
# 저장하기
result.to_excel("iherb_mctoil.xlsx", encoding='utf-8-sig')
# df.to_csv("blog_url.csv")

### 1.1.2 평균필터링 

In [37]:
mean_df = result.mean()
print(mean_df)

item_id    2.168254e+238
평점          4.508696e+00
평점수         6.281522e+02
dtype: float64


In [39]:
condition1 = result.평점  >= 4.51
condition2 = result.평점수 >= 628
df = result.loc[condition1, ['item_id','평점','평점수','리뷰url']]
df = result.loc[condition2, ['item_id','평점','평점수','리뷰url']]
print(df)
print("")
print("데이터길이: ",len(df))

   item_id   평점   평점수                                              리뷰url
0    99739  4.8  5969  https://kr.iherb.com/r/california-gold-nutriti...
2    66202  4.7  6447  https://kr.iherb.com/r/nature-s-way-organic-mc...
4    84966  4.7   719  https://kr.iherb.com/r/sports-research-organic...
5    71107  4.7  2215  https://kr.iherb.com/r/sports-research-mct-oil...
6    72853  4.6  1921  https://kr.iherb.com/r/now-foods-sports-mct-oi...
7    13376  4.8  3600  https://kr.iherb.com/r/now-foods-caprylic-acid...
10   13585  4.7   710  https://kr.iherb.com/r/now-foods-sports-mct-oi...
12   68705  4.6  1563  https://kr.iherb.com/r/jarrow-formulas-mct-oil...

데이터길이:  8


In [41]:
# 저장하기
df.to_excel("iherb_mctoil-filtered.xlsx", encoding='utf-8-sig')

## 1.2 유저리뷰 크롤링

In [2]:
#데이터로드 
df_load = pd.read_excel("iherb_mctoil-filtered.xlsx")
df_load

,Unnamed: 0,item_id,평점,평점수,리뷰url
0,0,99739,4.8,5960,https://kr.iherb.com/r/california-gold-nutriti...
1,2,66202,4.7,6443,https://kr.iherb.com/r/nature-s-way-organic-mc...
2,4,71107,4.7,2213,https://kr.iherb.com/r/sports-research-mct-oil...
3,5,72853,4.6,1918,https://kr.iherb.com/r/now-foods-sports-mct-oi...
4,6,13376,4.8,3598,https://kr.iherb.com/r/now-foods-caprylic-acid...
5,9,13585,4.7,708,https://kr.iherb.com/r/now-foods-sports-mct-oi...
6,11,68705,4.6,1562,https://kr.iherb.com/r/jarrow-formulas-mct-oil...
7,48,84966,4.7,716,https://kr.iherb.com/r/sports-research-organic...


In [3]:
df_load = df_load.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

df_load

,item_id,평점,평점수,리뷰url
0,99739,4.8,5960,https://kr.iherb.com/r/california-gold-nutriti...
1,66202,4.7,6443,https://kr.iherb.com/r/nature-s-way-organic-mc...
2,71107,4.7,2213,https://kr.iherb.com/r/sports-research-mct-oil...
3,72853,4.6,1918,https://kr.iherb.com/r/now-foods-sports-mct-oi...
4,13376,4.8,3598,https://kr.iherb.com/r/now-foods-caprylic-acid...
5,13585,4.7,708,https://kr.iherb.com/r/now-foods-sports-mct-oi...
6,68705,4.6,1562,https://kr.iherb.com/r/jarrow-formulas-mct-oil...
7,84966,4.7,716,https://kr.iherb.com/r/sports-research-organic...


In [13]:
%%time
# 수집할 글 갯수 정하기
user_list=[]
rev_count_list=[]


# 수집한 url 돌면서 데이터 수집

for k in tqdm_notebook(range(0,8)):

    # 리뷰창 띄우기
    url=df_load.리뷰url[k]
    driver.get(url)   
    time.sleep(2)

    for j in range(0,20):

        for i in range(0, 10):
            # 유저아이디링크따기
            userid =".customer-profile-link"
            user=driver.find_elements_by_css_selector(userid)
            userurl=user[i].get_attribute('href')
            user_list.append(userurl)

            #유저리뷰수 따기
            rev_count=driver.find_elements_by_css_selector(".snapshot-count")
            count= rev_count[i].text
            rev_count_list.append(count)

            if i == 9:
                pg_nx= driver.find_element_by_css_selector(".paging-btn-next")
                driver.execute_script("arguments[0].click();", pg_nx)
                time.sleep(2)
    print(df_load.item_id[k], len(user_list))

df_user=pd.DataFrame({"고객링크":user_list,"리뷰수":rev_count_list})   

  0%|          | 0/8 [00:00<?, ?it/s]

99739 200
66202 400
71107 600
72853 800
13376 1000
13585 1200
68705 1400
84966 1600
Wall time: 8min 50s


In [14]:
df_user

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
1,https://kr.iherb.com/me/4774927486492925388,6
2,https://kr.iherb.com/me/4811466204621279669,0
3,https://kr.iherb.com/me/4680214676756300470,513
4,https://kr.iherb.com/me/4923014602873352856,8
...,...,...
1595,https://kr.iherb.com/me/C2CE0F4095,0
1596,https://kr.iherb.com/me/81A6D19053,0
1597,https://kr.iherb.com/me/4159E88F23,0
1598,https://kr.iherb.com/me/DD9B1D0957,6


In [15]:
#중복제거
df_user=df_user.drop_duplicates(['고객링크'],keep = 'first')
df_user

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
1,https://kr.iherb.com/me/4774927486492925388,6
2,https://kr.iherb.com/me/4811466204621279669,0
3,https://kr.iherb.com/me/4680214676756300470,513
4,https://kr.iherb.com/me/4923014602873352856,8
...,...,...
1595,https://kr.iherb.com/me/C2CE0F4095,0
1596,https://kr.iherb.com/me/81A6D19053,0
1597,https://kr.iherb.com/me/4159E88F23,0
1598,https://kr.iherb.com/me/DD9B1D0957,6


### 1.2.1 유저리뷰 크롤링 전처리

In [16]:
#숫자화
df_user["리뷰수"]=pd.to_numeric(df_user["리뷰수"]) 

In [17]:
#리뷰수 10건이상필터링
df_user_10 = df_user.loc[(df_user.리뷰수>=10), ['고객링크','리뷰수']]
df_user_10

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
3,https://kr.iherb.com/me/4680214676756300470,513
8,https://kr.iherb.com/me/A22434FB52,16
10,https://kr.iherb.com/me/4883425644413566162,84
11,https://kr.iherb.com/me/5200700960892458955,32
...,...,...
1578,https://kr.iherb.com/me/5426883134385690579,10
1580,https://kr.iherb.com/me/D846920492,93
1583,https://kr.iherb.com/me/5614149106978046441,36
1588,https://kr.iherb.com/me/915515DB69,44


In [18]:
df_user.to_excel("iherb_userlist.xlsx", encoding='utf-8-sig')
df_user_10.to_excel("iherb_userlist_morethan10.xlsx", encoding='utf-8-sig')

## 1.3 유저 아이템 크롤링

In [3]:
#데이터로드 
df_load1 = pd.read_excel("iherb_userlist_morethan10.xlsx")
df_load1 = df_load1.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

df_load1

,고객링크,리뷰수
0,https://kr.iherb.com/me/aminochan,275
1,https://kr.iherb.com/me/4680214676756300470,512
2,https://kr.iherb.com/me/A22434FB52,16
3,https://kr.iherb.com/me/4883425644413566162,84
4,https://kr.iherb.com/me/5200700960892458955,32
...,...,...
570,https://kr.iherb.com/me/D846920492,93
571,https://kr.iherb.com/me/5614149106978046441,36
572,https://kr.iherb.com/me/4790040468155740968,44
573,https://kr.iherb.com/me/0479329787,23


In [4]:
len(df_load1)

575

In [5]:
%%time
# 수집할 글 갯수 정하기
rev_item_list=[]
rev_item_rating_list=[]
userid_list=[]


# 수집한 url 돌면서 데이터 수집
for k in tqdm_notebook(range(0,len(df_load1))):
    
    
    # 리뷰창 띄우기
    url=df_load1.고객링크[k]
    driver.get(url)   
    time.sleep(2)
    
    try:
        for j in range(0,100):
            for i in range(0, 10):
                # 글 띄우기

                rev_item=driver.find_elements_by_css_selector(".product-review-name")
                element1 =rev_item[i].text
                rev_item_list.append(element1)

                rev_item_rating=driver.find_elements_by_css_selector("div.col-xs-14.review-content > div > div.rating > a>svg>use")
                element2 =rev_item_rating[i].get_attribute("xlink:href")
                rev_item_rating_list.append(element2)

                userid_list.append(k)
                print(element2)
            
                if i == 9:
                    pg_nx= driver.find_element_by_css_selector(".pagination-right")
                    driver.execute_script("arguments[0].click();", pg_nx)
                    time.sleep(2)
                    
    except:
        print(k)

        
        
ratings=pd.DataFrame({"userid":userid_list,"item":rev_item_list, "rating":rev_item_rating_list})

  0%|          | 0/575 [00:00<?, ?it/s]

#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_20
#icon-stars_50
#icon-stars_30
#icon-star

#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_10
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-star

#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
16
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
17
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
18
#icon-stars_50
19
20
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
21
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-s

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-star

...

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
532
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_10
#icon-stars_40
#icon-stars_50
#icon-stars_20
#icon-stars_50
#icon-stars_50
533
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_40
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_20
#icon-stars_30
#icon-stars_20
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_20
#icon-stars_30
#icon-stars_30
#icon-stars_30
#icon-stars_30
#i

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
545
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-

#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_10
#icon-stars_10
#icon-stars_30
#icon-stars_10
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
552
#icon-stars_50
#icon-stars_50
#icon-stars_50
553
#icon-stars_40
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_20
#icon-stars_50
554
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_5

#icon-stars_50
565
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-

569
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_30
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_40
#icon-stars_40
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_40
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-stars_50
#icon-

In [ ]:
#저장하기
ratings.to_excel("iherb_ratings.xlsx", encoding='utf-8-sig')

In [ ]:
#데이터로드 
ratings = pd.read_excel("iherb_ratings.xlsx")
ratings = df_load2.drop("Unnamed: 0", axis=1)  # 불필요한 칼럼 삭제

ratings

In [7]:
rating= ratings["rating"].str.split('_').str[1].tolist()
ratings["rating"]=rating
ratings

,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",50
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",50
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",30
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",50
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",50
...,...,...,...
31591,574,"NOW Foods,비타민D-3, 50mcg(2,000IU), 소프트젤 120정",50
31592,574,"California Gold Nutrition,LactoBif 프로바이오틱스, 50...",50
31593,574,"California Gold Nutrition,Gold C, 비타민C, 1,000m...",50
31594,574,"Gerber,유기농 퍼프, 무화과 열매, 42g(1.48oz)",50


In [8]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31596 entries, 0 to 31595
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userid  31596 non-null  int64 
 1   item    31596 non-null  object
 2   rating  31596 non-null  object
dtypes: int64(1), object(2)
memory usage: 740.7+ KB


In [9]:
ratings["rating"]=pd.to_numeric(ratings["rating"])
ratings["rating"]=ratings["rating"]/10
ratings

,userid,item,rating
0,0,"Sierra Fit,프로틴 크리스프, 솔티드 토피, 12개입, 각 56g(1.98oz)",5.0
1,0,"Sierra Fit,프로틴 크리스프, 땅콩버터, 바 12개입, 각 56g(1.98oz)",5.0
2,0,"Sierra Fit,프로틴 크리스프, 초콜릿 칩 쿠키 도우, 12개입, 각 56g(...",3.0
3,0,"Sierra Fit,유청 단백질 컴플리트, 바닐라, 2.27kg(5lb)",5.0
4,0,"Bob's Red Mill,Organic Quick Cooking Rolled Oa...",5.0
...,...,...,...
31591,574,"NOW Foods,비타민D-3, 50mcg(2,000IU), 소프트젤 120정",5.0
31592,574,"California Gold Nutrition,LactoBif 프로바이오틱스, 50...",5.0
31593,574,"California Gold Nutrition,Gold C, 비타민C, 1,000m...",5.0
31594,574,"Gerber,유기농 퍼프, 무화과 열매, 42g(1.48oz)",5.0


In [10]:
#저장하기
ratings.to_excel("iherb_ratings.xlsx", encoding='utf-8-sig')

# 2. CF추천알고리즘 전처리

## 2.1 사용자-아이템 평점행렬전환

In [11]:
#행렬변환
ratings_matrix = ratings.pivot_table('rating', index='userid', columns='item')

print(ratings_matrix.shape)
ratings_matrix

(531, 11682)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 장용성 코팅 정제 180정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,NaN,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#nan값변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 장용성 코팅 정제 180정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,0.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 아이템간 유사도산출

In [13]:
# 아이템-사용자 행렬로 transpose 한다.
ratings_matrix_T = ratings_matrix.transpose()  # 전치 행렬

print(ratings_matrix_T.shape)
ratings_matrix_T.head(3)

(11682, 531)


userid,0,1,2,3,4,5,6,8,9,10,...,565,566,567,568,569,570,571,572,573,574
item,,,,,,,,,,,,,,,,,,,,,
"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)",4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)

print(item_sim_df.shape)
item_sim_df.head(3)

(11682, 11682)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 장용성 코팅 정제 180정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
item,,,,,,,,,,,,,,,,,,,,,
"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)",1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.242091,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
"1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)",0.0,1.0,1.0,0.312348,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)",0.0,1.0,1.0,0.312348,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.408248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 3. 아이템 기반 CF-KNN

In [15]:
# 평점 벡터(행 벡터)와 유사도 벡터(열 벡터)를 내적(dot)해서 예측 평점을 계산하는 함수 정의
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

In [17]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred

array([[0.18184704, 3.475089  , 3.475089  , ..., 0.18184704, 0.10432812,
        0.10432812],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00566674,
        0.00566674],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.00779714, 0.00779714, ..., 0.        , 0.00777344,
        0.00777344],
       [0.02553839, 0.00982678, 0.00982678, ..., 0.02553839, 0.01640422,
        0.01640422],
       [0.416065  , 0.05482336, 0.05482336, ..., 0.416065  , 0.37277881,
        0.37277881]])

In [18]:
# 데이터프레임으로 변환
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
print(ratings_pred_matrix.shape)
ratings_pred_matrix.head(10)

(531, 11682)


item,"1Kvit-C,비타민 C, 수분, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.22oz(6.20g)","1Kvit-C,비타민 C, 집중력, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 30팩, 각 0.23oz(6.40g)","1Kvit-C,비타민C, 에너지, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.80g(0.24oz)","1Kvit-C,비타민C, 에센셜, 발포 드링크 믹스, 천연 오렌지 맛, 1,000mg, 30팩, 각 6.90g(0.24oz)","1Kvit-C,비타민C, 필수, 발포 드링크 믹스, 천연 오렌지 향, 1,000mg, 3팩, 각 6.90g(0.24oz)","21st Century,600+D3 플러스 미네랄, 120정","21st Century,600+D3, 칼슘 & 비타민D3 보충제, 75정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 장용성 코팅 정제 180정","21st Century,Arthri-Flex 어드밴티지 + 비타민D3, 코팅 정제 120정","21st Century,B-100 복합체, 서방정, 타블렛 60정",...,"iHerb Goods,홍보용 메모 액세서리, 3 피스","iHerb Nutritionals,비타민 D3, 5,000 IU, 120 소프트젤","iUNIK,Propolis Vitamin Sleeping Mask, 60 ml","pHion Balance,애씨드 드레이니지, 캡슐 60정","petnc NATURAL CARE,알래스카 야생 연어 오일, 강아지용, 532ml(18oz)","petnc NATURAL CARE,엉덩이 및 관절, 모든 견종, 간, 소프트 츄 90정","petnc NATURAL CARE,클린 이어 클렌징 패드, 고양이 및 강아지용, 패드 90개","petnc NATURAL CARE,펫 내츄럴 케어, 시니어 종합 비타민 포뮬라, 시니어 도그, 간(Liver) 맛, 60 츄어블 정","theBalm Cosmetics,바하마 마마, 브론저, 섀도 & 컨투어 파우더, 0.25oz(7.08g)","theBalm Cosmetics,핫 마마, 섀도우/블러시, 0.25oz(7.08g)"
userid,,,,,,,,,,,,,,,,,,,,,
0,0.181847,3.475089,3.475089,1.488804,0.059627,0.036277,0.071539,0.026464,0.034909,0.067972,...,0.057039,2.376591,0.0,0.181847,0.198462,0.050864,0.198462,0.181847,0.104328,0.104328
1,0.000000,0.000000,0.000000,0.004157,0.003296,0.000000,0.000000,0.002689,0.000000,0.001249,...,0.002205,0.001585,0.0,0.000000,0.004420,0.006161,0.004420,0.000000,0.005667,0.005667
2,0.000000,0.000000,0.000000,0.000000,0.001799,0.000000,0.000000,0.005769,0.000000,0.004842,...,0.001423,0.001982,0.0,0.000000,0.005524,0.004765,0.005524,0.000000,0.000000,0.000000
3,0.045703,0.022471,0.022471,0.044205,0.039666,0.021131,0.053514,0.001510,0.054467,0.035294,...,0.041856,0.016945,0.0,0.045703,0.043158,0.048112,0.043158,0.045703,0.104860,0.104860
4,0.033064,0.007080,0.007080,0.007412,0.013222,0.045533,0.014353,0.000000,0.012875,0.014211,...,0.010408,0.006412,0.0,0.033064,0.003160,0.007184,0.003160,0.033064,0.003654,0.003654
5,0.041330,0.006743,0.006743,0.003515,0.002179,0.016566,0.006750,0.001842,0.000000,0.002629,...,0.005514,0.002267,0.0,0.041330,0.006985,0.007319,0.006985,0.041330,0.000000,0.000000
6,0.033064,0.005394,0.005394,0.005834,0.005117,0.000000,0.012842,0.000000,0.016200,0.001239,...,0.005534,0.001813,0.0,0.033064,0.000000,0.006323,0.000000,0.033064,0.022698,0.022698
8,0.036677,0.018866,0.018866,0.038263,0.019720,0.047213,0.021464,0.004082,0.020164,0.009913,...,0.013383,0.011349,0.0,0.036677,0.039954,0.016718,0.039954,0.036677,0.024123,0.024123
9,0.036264,0.026785,0.026785,0.010701,0.003604,0.000000,0.005923,0.000000,0.000000,0.008704,...,0.002693,0.021542,0.0,0.036264,0.014666,0.008976,0.014666,0.036264,0.010633,0.010633


## 3.1 예측 평점 정확도를 판단하기 위해 오차 함수인 RMSE를 이용

In [19]:
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


아이템 기반 모든 인접 이웃 MSE:  11.455933738377764


## 3.2 top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산

In [20]:
# 사용자 9번에게 영화를 추천해보자
# 추천에 앞서 9번 사용자가 높은 평점을 준 영화를 확인해보면
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

item
Badger Company,유기농 슬립 밤, 라벤더 & 베르가못, 56g(2oz)                             5.0
California Gold Nutrition,실리마린 복합체, 베지 캡슐 120정                            5.0
California Gold Nutrition,아스타잔틴, AstaLif 순수 아이슬란드산, 12mg, 베지 소프트젤 120정    5.0
California Gold Nutrition,풀 스펙트럼 비타민K2, 120mcg, 베지 캡슐 60정                 5.0
Doctor's Best,알파리포산 600, 600mg, 베지 캡슐 180정                                5.0
Kitchens of India,버터 치킨 커리 페이스트, 농축 소스, 마일드, 100g(3.5oz)                  5.0
NOW Foods,비타민A, 25,000IU, 소프트젤 250정                                       5.0
Sea Tangle Noodle Company,켈프 누들, 녹차 함유, 12 온스 (340g)                      5.0
Sports Research,오메가3 피쉬 오일, 트리플 스트렝스, 1,250mg, 소프트젤 180정                  5.0
Walden Farms,마리나라 소스, 토마토 바질, 340g(12oz)                                  5.0
Name: 9, dtype: float64

In [21]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred


In [22]:
# 실행시간 2분 정도 걸림
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))

# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

아이템 기반 인접 TOP-20 이웃 MSE:  7.411762916099533


## 3.3 사용자 상품 추천

### 사용자가 미사용한 상품 추천

In [23]:
# 사용자가 미사용한 상품 추천user_rating이 0보다 크면 기존에 사용한상품 
def get_unused_item(ratings_matrix, userid):
    
    user_rating = ratings_matrix.loc[userid,:]
    already_used = user_rating[ user_rating > 0].index.tolist()
    
    items_list = ratings_matrix.columns.tolist()
    unused_list = [ item for item in items_list if item not in already_used]
    
    return unused_list


In [24]:
def recomm_item_by_userid(pred_df, userid, unused_list, top_n=10):

    recomm_items = pred_df.loc[userid, unused_list].sort_values(ascending=False)[:top_n]
    return recomm_items

In [25]:
unused_list = get_unused_item(ratings_matrix, 9)
recomm_items = recomm_item_by_userid(ratings_pred_matrix, 9, unused_list, top_n=10)
recomm_items = pd.DataFrame(data=recomm_items.values,index=recomm_items.index, columns=['pred_score'])
recomm_items

,pred_score
item,
"NatureWise,유기농 강황 커큐민, 베지 캡슐 180정",0.318805
"Annie's Naturals,유기농 케첩, 680g(24oz)",0.300946
"Sir Kensington's,아보카도 오일 마요네즈, 473ml(16fl oz)",0.259400
"Kitchens of India,치킨 커리 페이스트, 농축 소스, 중간, 100g(3.5oz)",0.257231
"Source Naturals,Chromemate 크로뮴 GTF, 200mcg, 240정",0.256203
"ChocZero,Maple Syrup, Sugar Free, 12 oz",0.249577
"Quest Nutrition,토르티야 스타일 단백질 칩, 나초 치즈, 12봉, 각 32g(1.1oz)",0.246900
"Solaray,칼슘 마그네슘 2:1 비율, 베지 캡슐 180정",0.243209
"NOW Foods,영양 효모 플레이크, 4.5 oz(128 g)",0.242032


In [26]:
unused_list = get_unused_item(ratings_matrix, 345)
recomm_items = recomm_item_by_userid(ratings_pred_matrix, 345, unused_list, top_n=15)
recomm_items = pd.DataFrame(data=recomm_items.values,index=recomm_items.index, columns=['pred_score'])
recomm_items

,pred_score
item,
"Nature's Bounty,멜라토닌, 1 밀리그램, 180 정",0.332698
"Nuby,Teethe Eez, 수딩 티더, 3개월 이상, 블루",0.332698
"Natrol,글루코사민, 히알루론 산 & MSM , 90 캡슐",0.332698
"Jarrow Formulas,PQQ, 10mg, 캡슐 30정",0.332698
"NOW Foods,호박씨오일, 1,000mg, 소프트젤 200정",0.275969
"California Gold Nutrition,비타민D3, 125mcg(5,000IU), 피쉬 젤라틴 소프트젤 90정",0.267296
"Solgar,비타민D3가 함유된 액상 칼슘 마그네슘 시트레이트, 천연 블루베리 맛, 473ml(16fl oz)",0.266033
"California Gold Nutrition,CollagenUP, 저분자 해양 콜라겐 + 히알루론산 + 비타민C, 무맛, 206g(7.26oz)",0.259916
"Naturade,Total Soy, 체중 감량 셰이크, 초콜릿, 540g(1.2lbs)",0.255241
